In [1]:
import pandas as pd
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, precision_recall_curve, average_precision_score, mean_squared_error, r2_score, mean_absolute_error
from scipy.stats import pearsonr

import numpy as np

# preds

# Load the training set of meta-model
bbbp_chemberta2_valid2 = pd.read_csv('./chemberta2/results/bbbp/chemberta2_valid2_bbbp_2_predictions.csv')
bbbp_molformer_valid2 = pd.read_csv('./molformer/results/bbbp/molformer_valid2_bbbp_2_epoch29.csv')
bbbp_molbert_valid2 = pd.read_csv('./molbert/results/bbbp/molbert_valid2_bbbp_2.csv')

# Load the test data for each model
bbbp_chemberta2_test = pd.read_csv('./chemberta2/results/bbbp/chemberta2_test_bbbp_2_predictions.csv')
bbbp_molformer_test = pd.read_csv('./molformer/results/bbbp/molformer_test_bbbp_2_epoch29.csv')
bbbp_molbert_test = pd.read_csv('./molbert/results/bbbp/molbert_test_bbbp_2.csv')

# features

# Load the features from chemberta
bbbp_chemberta2_features_valid2 = pd.read_csv('./chemberta2/features/bbbp/chemberta2_valid2_bbbp_2_features.csv')
bbbp_chemberta2_features_test = pd.read_csv('./chemberta2/features/bbbp/chemberta2_test_bbbp_2_features.csv')

# Load the features from molformer
bbbp_molformer_features_valid2 = pd.read_csv('./molformer/features/bbbp/molformer_valid2_bbbp_2_features.csv')
bbbp_molformer_features_test = pd.read_csv('./molformer/features/bbbp/molformer_test_bbbp_2_features.csv')

# Load the features from molbert
bbbp_molbert_features_valid2 = pd.read_csv('./molbert/features/bbbp/molbert_valid2_bbbp_2_features.csv')
bbbp_molbert_features_test = pd.read_csv('./molbert/features/bbbp/molbert_test_bbbp_2_features.csv')

In [2]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, average_precision_score

# Preparing the actual and predicted values
# Chemberta2
bbbp_chemberta_actual = bbbp_chemberta2_test['p_np']
bbbp_chemberta_pred = bbbp_chemberta2_test['y_pred']
bbbp_chemberta_probs = bbbp_chemberta2_test[['softmax_class_0_prob', 'softmax_class_1_prob']]

# Molformer
bbbp_molformer_actual = bbbp_molformer_test['Actual']
bbbp_molformer_pred = (bbbp_molformer_test['Prob_Class_1'] > 0.5).astype(int)
bbbp_molformer_probs = bbbp_molformer_test[['Prob_Class_0', 'Prob_Class_1']]

# Molbert
bbbp_molbert_actual = bbbp_molbert_test['target']
bbbp_molbert_pred = bbbp_molbert_test['pred']
bbbp_molbert_probs = bbbp_molbert_test['prob']

# Calculating metrics
bbbp_metrics_results = {}

for model_name, actual, pred, probs in [("Chemberta2", bbbp_chemberta_actual, bbbp_chemberta_pred, bbbp_chemberta_probs['softmax_class_1_prob']),
                                         ("Molformer", bbbp_molformer_actual, bbbp_molformer_pred, bbbp_molformer_probs['Prob_Class_1']),
                                         ("Molbert", bbbp_molbert_actual, bbbp_molbert_pred, bbbp_molbert_probs)]:
    bbbp_metrics_results[model_name] = {
        "Accuracy": accuracy_score(actual, pred),
        "F1 Score": f1_score(actual, pred),
        "ROC-AUC": roc_auc_score(actual, probs),
        "PR-AUC": average_precision_score(actual, probs)
    }

bbbp_metrics_results

{'Chemberta2': {'Accuracy': 0.8634146341463415,
  'F1 Score': 0.9072847682119205,
  'ROC-AUC': 0.9566584632212196,
  'PR-AUC': 0.9881838964094798},
 'Molformer': {'Accuracy': 0.9264705882352942,
  'F1 Score': 0.9538461538461539,
  'ROC-AUC': 0.9208172812328014,
  'PR-AUC': 0.9642320592298499},
 'Molbert': {'Accuracy': 0.9411764705882353,
  'F1 Score': 0.9625,
  'ROC-AUC': 0.9640891579526691,
  'PR-AUC': 0.9894812187118154}}

In [3]:
# Identify the 'smiles' values in chemberta2_valid2 that are not in molbert_valid2
missing_smiles_molformer_valid2 = set(bbbp_chemberta2_valid2['smiles']) - set(bbbp_molformer_valid2['smiles'])
print(f"Missing smiles in molformer_valid2: {missing_smiles_molformer_valid2}")

# Identify the 'smiles' values in chemberta2_valid2 that are not in molbert_valid2
missing_smiles_molbert_valid2 = set(bbbp_chemberta2_valid2['smiles']) - set(bbbp_molbert_valid2['smiles'])
print(f"Missing smiles in molbert_valid2: {missing_smiles_molbert_valid2}")

# Combine the invalid indices from molformer_valid2 with the missing indices from molbert_valid2
combined_invalid_smiles_valid2 = list(set(missing_smiles_molformer_valid2).union(set(missing_smiles_molbert_valid2)))

print(f"These indices will be removed from the valid2 set: {combined_invalid_smiles_valid2}")

Missing smiles in molformer_valid2: {'s1cc(CSCCN\\C(NC)=[NH]\\C#N)nc1\\[NH]=C(\\N)N', 'c1(nc(NC(N)=[NH2])sc1)CSCCNC(=[NH]C#N)NC', 'c1c(c(ncc1)CSCCN\\C(=[NH]\\C#N)NCC)Br'}
Missing smiles in molbert_valid2: {'c1c(c(ncc1)CSCCN\\C(=[NH]\\C#N)NCC)Br', '[Na].CO[C@]1(NC(=O)CSC(F)(F)F)[C@H]2OCC(=C(N2C1=O)C(O)=O)CSc3nnnn3CCO', '[Na+].CC(=O)Nc1c(I)c(NC(C)=O)c(I)c(C([O-])=O)c1I', '[Na+].[Na+].[Na+].[O-]C(=O)[P]([O-])([O-])=O', '[Na+].[Na+].CC1(C)S[C@@H]2[C@H](NC(=O)[C@H](N[S]([O-])(=O)=O)c3ccccc3)C(=O)N2[C@H]1C([O-])=O', 's1cc(CSCCN\\C(NC)=[NH]\\C#N)nc1\\[NH]=C(\\N)N', 'c1(nc(NC(N)=[NH2])sc1)CSCCNC(=[NH]C#N)NC', '[Na+].CCOc1ccc2ccccc2c1C(=O)N[C@H]3[C@H]4SC(C)(C)[C@@H](N4C3=O)C([O-])=O', '[Na+].COCC1=C(N2[C@H](SC1)[C@H](NC(=O)C(=N/OC)\\c3csc(N)n3)C2=O)C([O-])=O', '[Na+].[O-][S](=O)(=O)CCS'}
These indices will be removed from the valid2 set: ['c1c(c(ncc1)CSCCN\\C(=[NH]\\C#N)NCC)Br', '[Na].CO[C@]1(NC(=O)CSC(F)(F)F)[C@H]2OCC(=C(N2C1=O)C(O)=O)CSc3nnnn3CCO', '[Na+].CC(=O)Nc1c(I)c(NC(C)=O)c(I)c(C([O-])=

In [4]:
# Function to remove invalid SMILES
def remove_invalid_smiles(df, invalid_smiles_list):
    return df[~df['smiles'].isin(invalid_smiles_list)]

# Remove invalid SMILES from each dataframe
bbbp_chemberta2_valid2 = remove_invalid_smiles(bbbp_chemberta2_valid2, combined_invalid_smiles_valid2)
bbbp_molformer_valid2 = remove_invalid_smiles(bbbp_molformer_valid2, combined_invalid_smiles_valid2)
bbbp_molbert_valid2 = remove_invalid_smiles(bbbp_molbert_valid2, combined_invalid_smiles_valid2)
bbbp_chemberta2_features_valid2 = remove_invalid_smiles(bbbp_chemberta2_features_valid2, combined_invalid_smiles_valid2)
bbbp_molformer_features_valid2 = remove_invalid_smiles(bbbp_molformer_features_valid2, combined_invalid_smiles_valid2)
bbbp_molbert_features_valid2 = remove_invalid_smiles(bbbp_molbert_features_valid2, combined_invalid_smiles_valid2)

# Print the shapes of the dataframes after removal
print(bbbp_chemberta2_valid2.shape)
print(bbbp_molformer_valid2.shape)
print(bbbp_molbert_valid2.shape)
print(bbbp_chemberta2_features_valid2.shape)
print(bbbp_molformer_features_valid2.shape)
print(bbbp_molbert_features_valid2.shape)

(400, 8)
(400, 6)
(400, 4)
(400, 386)
(400, 769)
(400, 769)


In [5]:
bbbp_y_ensemble_valid2 = bbbp_chemberta2_valid2['p_np']

# Convert the ensemble target to a Series if not already done
bbbp_y_ensemble_valid2_s = pd.Series(bbbp_y_ensemble_valid2).reset_index(drop=True)

# Create dataframes for each model's class 1 probability
bbbp_chemberta2_prob = pd.DataFrame({'chemberta2': bbbp_chemberta2_valid2['softmax_class_1_prob']})
bbbp_chemberta2_prob.reset_index(drop=True, inplace=True)

bbbp_molformer_prob = pd.DataFrame({'molformer': bbbp_molformer_valid2['Prob_Class_1']})
bbbp_molformer_prob.reset_index(drop=True, inplace=True)

bbbp_molbert_prob = pd.DataFrame({'molbert': bbbp_molbert_valid2['prob']})
bbbp_molbert_prob.reset_index(drop=True, inplace=True)

# do the same for features bbbp_chemberta2_features_valid2.iloc[:, 2:]
bbbp_chemberta2_features = pd.DataFrame(bbbp_chemberta2_features_valid2.iloc[:, 2:])
bbbp_chemberta2_features.reset_index(drop=True, inplace=True)

bbbp_molformer_features = pd.DataFrame(bbbp_molformer_features_valid2.iloc[:, 1:])
bbbp_molformer_features.reset_index(drop=True, inplace=True)

bbbp_molbert_features = pd.DataFrame(bbbp_molbert_features_valid2.iloc[:, 1:])
bbbp_molbert_features.reset_index(drop=True, inplace=True)

# bbbp_features = pd.concat([bbbp_chemberta2_features, bbbp_molformer_features, bbbp_molbert_features], axis=1)

# Combine probabilities into one dataframe
train_bbbp_prob = pd.concat([bbbp_chemberta2_prob, bbbp_molformer_prob, bbbp_molbert_prob], axis=1)

# Function to calculate BCE for each row with epsilon to avoid log(0)
def calculate_bce_rowwise(y_true, y_pred, epsilon=1e-12):
    # Clip y_pred to ensure it's between epsilon and 1-epsilon
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    return -(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

# Calculate row-wise BCE for each model
bce_chemberta = calculate_bce_rowwise(bbbp_y_ensemble_valid2_s, bbbp_chemberta2_prob['chemberta2'])
bce_molformer = calculate_bce_rowwise(bbbp_y_ensemble_valid2_s, bbbp_molformer_prob['molformer'])
bce_molbert = calculate_bce_rowwise(bbbp_y_ensemble_valid2_s, bbbp_molbert_prob['molbert'])

# Create a dataframe for row-wise BCE losses
bce_loss_df = pd.DataFrame({
    'bce_chemberta': bce_chemberta,
    'bce_molformer': bce_molformer,
    'bce_molbert': bce_molbert
})

# Final ensemble X matrix: Combine row-wise BCE losses, predictions, and features
bbbp_X_ensemble_valid2 = pd.concat([bce_loss_df, train_bbbp_prob], axis=1)

In [6]:
import numpy as np
import pandas as pd
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

# Set seed for reproducibility
seed = 0

# Combine probabilities with their respective feature sets
chemberta_X = pd.concat([bbbp_chemberta2_prob, bbbp_chemberta2_features], axis=1)
molformer_X = pd.concat([bbbp_molformer_prob, bbbp_molformer_features], axis=1)
molbert_X = pd.concat([bbbp_molbert_prob, bbbp_molbert_features], axis=1)

# Standardize each dataset
scaler_chemberta = StandardScaler().fit(chemberta_X)
scaler_molformer = StandardScaler().fit(molformer_X)
scaler_molbert = StandardScaler().fit(molbert_X)

chemberta_X_scaled = scaler_chemberta.transform(chemberta_X)
molformer_X_scaled = scaler_molformer.transform(molformer_X)
molbert_X_scaled = scaler_molbert.transform(molbert_X)

# Define the binary cross-entropy loss values as target variables (y)
chemberta_y_bce = bce_chemberta  # Row-wise BCE loss calculated earlier
molformer_y_bce = bce_molformer  # Row-wise BCE loss calculated earlier
molbert_y_bce = bce_molbert      # Row-wise BCE loss calculated earlier

# Initialize the ElasticNet models with a random_state for reproducibility
elastic_net_chemberta = ElasticNet(max_iter=10000, tol=0.0001, random_state=seed)
elastic_net_molformer = ElasticNet(max_iter=10000, tol=0.0001, random_state=seed)
elastic_net_molbert = ElasticNet(max_iter=10000, tol=0.0001, random_state=seed)

# Setup cross-validation for alpha and l1_ratio tuning
param_grid = {
    'alpha': np.logspace(-4, 1, 10),
    'l1_ratio': np.linspace(0, 1, 10)  # l1_ratio between 0 (Ridge) and 1 (Lasso)
}

# Set up GridSearchCV for ElasticNet with neg_mean_squared_error scoring and random_state
cv_chemberta = GridSearchCV(estimator=elastic_net_chemberta, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
cv_molformer = GridSearchCV(estimator=elastic_net_molformer, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
cv_molbert = GridSearchCV(estimator=elastic_net_molbert, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)

# Fit GridSearchCV for each dataset with BCE as the target
cv_chemberta.fit(chemberta_X_scaled, chemberta_y_bce)
cv_molformer.fit(molformer_X_scaled, molformer_y_bce)
cv_molbert.fit(molbert_X_scaled, molbert_y_bce)

# Retrieve the best models and parameters
best_model_chemberta = cv_chemberta.best_estimator_
best_model_molformer = cv_molformer.best_estimator_
best_model_molbert = cv_molbert.best_estimator_

/Users/lorrainelu/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.194e+00, tolerance: 6.423e-03 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/Users/lorrainelu/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.034e+00, tolerance: 6.282e-03 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers 

In [7]:
# identify missing in test
missing_smiles_molformer_test = set(bbbp_chemberta2_test['smiles']) - set(bbbp_molformer_test['smiles'])
print(f"Missing smiles in molformer_test: {missing_smiles_molformer_test}")

missing_smiles_molbert_test = set(bbbp_chemberta2_test['smiles']) - set(bbbp_molbert_test['smiles'])
print(f"Missing smiles in molbert_test: {missing_smiles_molbert_test}")

# Combine the invalid smiles from molformer_test with the missing smiles from molbert_test
combined_invalid_smiles_test = list(set(missing_smiles_molformer_test).union(set(missing_smiles_molbert_test)))
print(f"These indices will be removed from the test set: {combined_invalid_smiles_test}")

Missing smiles in molformer_test: {'Cc1nc(sc1)\\[NH]=C(\\N)N'}
Missing smiles in molbert_test: {'Cc1nc(sc1)\\[NH]=C(\\N)N'}
These indices will be removed from the test set: ['Cc1nc(sc1)\\[NH]=C(\\N)N']


In [8]:
# Remove invalid SMILES from each dataframe
bbbp_chemberta2_test = remove_invalid_smiles(bbbp_chemberta2_test, combined_invalid_smiles_test)
bbbp_molformer_test = remove_invalid_smiles(bbbp_molformer_test, combined_invalid_smiles_test)
bbbp_molbert_test = remove_invalid_smiles(bbbp_molbert_test, combined_invalid_smiles_test)
bbbp_chemberta2_features_test = remove_invalid_smiles(bbbp_chemberta2_features_test, combined_invalid_smiles_test)
bbbp_molformer_features_test = remove_invalid_smiles(bbbp_molformer_features_test, combined_invalid_smiles_test)
bbbp_molbert_features_test = remove_invalid_smiles(bbbp_molbert_features_test, combined_invalid_smiles_test)

# Print the shapes of the dataframes after removal
print(bbbp_chemberta2_test.shape)
print(bbbp_molformer_test.shape)
print(bbbp_molbert_test.shape)
print(bbbp_chemberta2_features_test.shape)
print(bbbp_molformer_features_test.shape)
print(bbbp_molbert_features_test.shape)

(204, 8)
(204, 6)
(204, 4)
(204, 386)
(204, 769)
(204, 769)


In [9]:
import numpy as np
from sklearn.metrics import log_loss

# Test data for each model
bbbp_chemberta2_prob_test = pd.DataFrame({'chemberta2': bbbp_chemberta2_test['softmax_class_1_prob']})
bbbp_chemberta2_prob_test.reset_index(drop=True, inplace=True)

bbbp_molformer_prob_test = pd.DataFrame({'molformer': bbbp_molformer_test['Prob_Class_1']})
bbbp_molformer_prob_test.reset_index(drop=True, inplace=True)

bbbp_molbert_prob_test = pd.DataFrame({'molbert': bbbp_molbert_test['prob']})
bbbp_molbert_prob_test.reset_index(drop=True, inplace=True)

bbbp_chemberta2_features_t = pd.DataFrame(bbbp_chemberta2_features_test.iloc[:, 2:])
bbbp_chemberta2_features_t.reset_index(drop=True, inplace=True)

bbbp_molformer_features_t  = pd.DataFrame(bbbp_molformer_features_test.iloc[:, 1:])
bbbp_molformer_features_t.reset_index(drop=True, inplace=True)

bbbp_molbert_features_t = pd.DataFrame(bbbp_molbert_features_test.iloc[:, 1:])
bbbp_molbert_features_t.reset_index(drop=True, inplace=True)

# Combine probabilities with the respective feature sets for the test set
chemberta_X_test = pd.concat([bbbp_chemberta2_prob_test, bbbp_chemberta2_features_t], axis=1)
molformer_X_test = pd.concat([bbbp_molformer_prob_test, bbbp_molformer_features_t], axis=1)
molbert_X_test = pd.concat([bbbp_molbert_prob_test, bbbp_molbert_features_t], axis=1)

# Standardize the test set based on the previously fitted scalers
chemberta_X_test_scaled = scaler_chemberta.transform(chemberta_X_test)
molformer_X_test_scaled = scaler_molformer.transform(molformer_X_test)
molbert_X_test_scaled = scaler_molbert.transform(molbert_X_test)

# Predict using the best models from valid2
chemberta_pred_test = best_model_chemberta.predict(chemberta_X_test_scaled)
molformer_pred_test = best_model_molformer.predict(molformer_X_test_scaled)
molbert_pred_test = best_model_molbert.predict(molbert_X_test_scaled)

# Convert the predictions (numpy arrays) to pandas Series
chemberta_pred_test_series = pd.Series(chemberta_pred_test, name='bce_chemberta')
molformer_pred_test_series = pd.Series(molformer_pred_test, name='bce_molformer')
molbert_pred_test_series = pd.Series(molbert_pred_test, name='bce_molbert')

# Now concatenate the series with the test set probabilities
bbbp_X_ensemble_test = pd.concat([
    chemberta_pred_test_series,                     # BCE for Chemberta
    molformer_pred_test_series,                     # BCE for Molformer
    molbert_pred_test_series,                       # BCE for Molbert
    bbbp_chemberta2_prob_test['chemberta2'],        # Chemberta test probabilities
    bbbp_molformer_prob_test['molformer'],          # Molformer test probabilities
    bbbp_molbert_prob_test['molbert']               # Molbert test probabilities
], axis=1)

bbbp_X_ensemble_test.columns = ['bce_chemberta', 'bce_molformer', 'bce_molbert', 'chemberta2', 'molformer', 'molbert']

# optional for evaluation
bbbp_y_ensemble_test = bbbp_chemberta2_test['p_np']

In [10]:
# use standard scaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
bbbp_X_ensemble_valid2_scaled = scaler.fit_transform(bbbp_X_ensemble_valid2)
bbbp_X_ensemble_test_scaled = scaler.transform(bbbp_X_ensemble_test)

# transform back to dataframe
bbbp_X_ensemble_valid2_scaled = pd.DataFrame(bbbp_X_ensemble_valid2_scaled, columns=bbbp_X_ensemble_valid2.columns)
bbbp_X_ensemble_test_scaled = pd.DataFrame(bbbp_X_ensemble_test_scaled, columns=bbbp_X_ensemble_test.columns)


In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, average_precision_score

# Define the model with elasticnet penalty
elastic_net_model = LogisticRegression(penalty='elasticnet', solver='saga', max_iter=5000, random_state=0)

# Use fewer discrete values for alpha and l1_ratio
alphas = [0.01, 0.1, 1, 3]  # Reduced number of points focusing on lower and mid-range
l1_ratios = [0.1, 0.5, 0.9]  # Reduced to three points, emphasizing edges and midpoint

# Convert alphas to Cs for the parameter grid (since C is the inverse of alpha)
Cs = [1/alpha for alpha in alphas]

# Create a more concise grid search using 5-fold cross-validation
params = {
    'C': Cs,
    'l1_ratio': l1_ratios
}

grid_search = GridSearchCV(elastic_net_model, param_grid=params, cv=5, scoring='roc_auc')

# Fit the grid search to the data
grid_search.fit(bbbp_X_ensemble_valid2_scaled, bbbp_y_ensemble_valid2)

# Best model after grid search
bbbp_best_elastic_model = grid_search.best_estimator_
print(grid_search.best_params_)

# Predict the test set
bbbp_elastic_pred = bbbp_best_elastic_model.predict(bbbp_X_ensemble_test_scaled)
bbbp_elastic_probs = bbbp_best_elastic_model.predict_proba(bbbp_X_ensemble_test_scaled)[:, 1]

# Calculate the metrics
bbbp_elastic_metrics = {
    "Accuracy": accuracy_score(bbbp_y_ensemble_test, bbbp_elastic_pred),
    "F1 Score": f1_score(bbbp_y_ensemble_test, bbbp_elastic_pred),
    "ROC-AUC": roc_auc_score(bbbp_y_ensemble_test, bbbp_elastic_probs),
    "PR-AUC": average_precision_score(bbbp_y_ensemble_test, bbbp_elastic_probs)
}

bbbp_elastic_metrics

{'C': 0.3333333333333333, 'l1_ratio': 0.1}


{'Accuracy': 0.9558823529411765,
 'F1 Score': 0.9719626168224299,
 'ROC-AUC': 0.9741331865712712,
 'PR-AUC': 0.9927232315576004}

In [12]:
# report all the metrics for bbbp
bbbp_metrics_results["Elastic Net"] = bbbp_elastic_metrics

bbbp_metrics_df = pd.DataFrame(bbbp_metrics_results).T

# keep 3 digits after the decimal point
bbbp_metrics_df = bbbp_metrics_df.round(3)

# export as csv
bbbp_metrics_df.to_csv('./split2_bbbp_metrics_elastic.csv')